# Motor Vehicle Accident XML Check Part 3 - Check Against TPS

This notebook checks the integrity of the periodic Motor Vehicle Accident (MVA) report XML files we receive into `\\tssrv7\CollisionsProcessed\AtSceneXmlBackups` against both the Oracle database and a dump of GO numbers and years from TPS's Versadex database (provided 2021-05-12 by Andrew Thompson).

In [1]:
%matplotlib inline

import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import configparser
import pathlib
import psycopg2
import psycopg2.extras as ppg2extras
import cx_Oracle
import sqlalchemy as sqla
import re

# Default mpl colours.
prop_cycle = plt.rcParams['axes.prop_cycle']
mpl_def_colors = prop_cycle.by_key()['color']

In [2]:
config = configparser.ConfigParser()
config.read(pathlib.Path.home().joinpath('.charlesconfig').as_posix());
postgres_settings = config['POSTGRES']
oracle_settings = config['ORACLE']

## Load XML Filenames

In [3]:
rootfolder = '/mnt/ldrive/AtSceneXmlBackups/'
xml_folders = os.listdir(rootfolder)
filenames = {fn: os.listdir(rootfolder + fn) for fn in xml_folders}

In [4]:
def get_name_info(filename, filepath):
    repyear_str, go_number_str = re.split("_|-", filename[:-4])[-2:]
    try:
        go_number = int(go_number_str)
    except Exception:
        raise ValueError("{0} was converted to {1}, which can't be "
                         "converted to an integer."
                         .format(mystr, go_number))

    try:
        repyear = int(repyear_str[2:])
        assert (repyear // 1000 > 1), ""
    except Exception as exc:
        raise ValueError("{0} was converted to {1}, which can't be "
                         "converted to an integer. Error raised:"
                         .format(mystr, repyear, str(exc)))

    acc_number = 1000000000 * int(repyear_str[-1]) + go_number
    
    return {'fullpath': filepath + "/" + filename,
            'filename': filename,
            'repyear': repyear,
            'go_number': go_number,
            'acc_number': acc_number}

coln_xml_r = [get_name_info(x, fn_key)
              for fn_key in filenames.keys() for x in filenames[fn_key]
              if x[-4:] == '.xml']
coln_xml_r = pd.DataFrame(coln_xml_r)

In [5]:
coln_xml_r_unique = coln_xml_r.groupby(
    ['filename', 'repyear', 'go_number', 'acc_number'])['fullpath'].count()
dups = coln_xml_r_unique[coln_xml_r_unique > 1]
dups = dups.reset_index()['acc_number'].unique()

coln_xml = coln_xml_r[['repyear', 'go_number', 'acc_number']].drop_duplicates()
coln_xml.reset_index(drop=True, inplace=True)

In [6]:
coln_xml_20152019 = coln_xml.loc[(coln_xml['repyear'] >= 2014) & (coln_xml['repyear'] < 2020), :].copy()
coln_xml_20152019.columns = ['year_reported_xml', 'go_num_xml', 'accnb_xml']

In [7]:
coln_xml_20152019

,year_reported_xml,go_num_xml,accnb_xml
1537,2014,1008236,4001008236
1538,2014,1057534,4001057534
1539,2014,1059448,4001059448
1540,2014,10667333,4010667333
1541,2014,1071106,4001071106
...,...,...,...
93576,2019,212276,9000212276
93638,2019,1961849,9001961849
93639,2019,2497348,9002497348
93769,2019,1205265,9001205265


## Load ACC Numbers from Oracle

In [8]:
sql_query = """SELECT ACCNB,
       EXTRACT(YEAR FROM ACCDATE) ACCYEAR,
       ACCDATE,
       CASE
         WHEN ACCNB >= 1000000000 THEN 'TPS'
         WHEN ACCNB >= 100000000 THEN 'CRC'
         ELSE 'DUNNO'
       END data_source
FROM TRAFFIC.ACC
WHERE accdate BETWEEN TO_DATE('2014-01-01', 'YYYY-MM-DD') AND TO_DATE('2019-12-31', 'YYYY-MM-DD')
GROUP BY ACCNB, ACCDATE"""

with cx_Oracle.connect(oracle_settings['user'], oracle_settings['password'],
                       "{host}:{port}/{service_name}".format(**oracle_settings),
                       encoding="UTF-8") as conn:
    df_acc = pd.read_sql(sql_query, con=conn)

In [9]:
sql_query = """SELECT ACCNB,
       EXTRACT(YEAR FROM ACCDATE) ACCYEAR,
       ACCDATE,
       CASE
         WHEN ACCNB >= 1000000000 THEN 'TPS'
         WHEN ACCNB >= 100000000 THEN 'CRC'
         ELSE 'DUNNO'
       END data_source
FROM TRAFFIC.ACC_ARCHIVE
WHERE accdate BETWEEN TO_DATE('2014-01-01', 'YYYY-MM-DD') AND TO_DATE('2019-12-31', 'YYYY-MM-DD')
GROUP BY ACCNB, ACCDATE"""

with cx_Oracle.connect(oracle_settings['user'], oracle_settings['password'],
                       "{host}:{port}/{service_name}".format(**oracle_settings),
                       encoding="UTF-8") as conn:
    df_accarchive = pd.read_sql(sql_query, con=conn)

In [10]:
df_acc['TABLE'] = 'ACC'
df_accarchive['TABLE'] = 'ACC_ARCHIVE'

In [11]:
df_oracle = pd.concat([df_acc, df_accarchive])
df_oracle.reset_index(inplace=True, drop=True)
df_oracle.columns = ['accnb_ocl', 'year_ocl', 'date_ocl', 'source_ocl', 'table_ocl']
df_oracle['accnb_ocl'] = df_oracle['accnb_ocl'].astype(int)
df_oracle

,accnb_ocl,year_ocl,date_ocl,source_ocl,table_ocl
0,267847,2019,2019-12-31,DUNNO,ACC
1,142586405,2014,2014-07-28,CRC,ACC
2,142588079,2014,2014-07-28,CRC,ACC
3,142937129,2014,2014-09-18,CRC,ACC
4,143084786,2014,2014-10-11,CRC,ACC
...,...,...,...,...,...
381038,9001398262,2019,2019-07-25,TPS,ACC_ARCHIVE
381039,9001402604,2019,2019-07-25,TPS,ACC_ARCHIVE
381040,9001469594,2019,2019-08-04,TPS,ACC_ARCHIVE
381041,8002261147,2018,2018-12-09,TPS,ACC_ARCHIVE


Something I didn't account for before was the existence of collisions with the same ACC number and year but nevertheless have a different date of occurrence, such as `148040265` and `168000033`. In other cases, the same ACC number shows up twice - once in `ACC` and once in `ACC_ARCHIVE` - see `168014482`, `4003077444`, `5000247160`, `6000443794`. For the sake of this analysis, let's drop one of the two rows.

In [12]:
count_dups = df_oracle.groupby(['accnb_ocl', 'year_ocl'])['accnb_ocl'].count()
count_dups[count_dups > 1].shape

(22,)

In [13]:
# Check that when we drop duplicates we get rid of 22 rows:
df_oracle.shape[0] - df_oracle.drop_duplicates(subset=['accnb_ocl', 'year_ocl']).shape[0]

22

In [14]:
df_oracle.drop_duplicates(subset=['accnb_ocl', 'year_ocl'], inplace=True)

## Load TPS Versadex Data

In [15]:
# Don't read in column 2 since everything comes from Versadex.
df_vdx = pd.read_csv("/mnt/c/Users/czhu5/Documents/Ubuntu/All VDX MVC 2014-2019.csv",
                     usecols=[0, 2, 3, 4], parse_dates=[2,])
df_vdx.columns = ['go_id_vdx', 'go_num_vdx', 'date_vdx', 'year_vdx']
df_vdx['year_reported_vdx'] = df_vdx['go_id_vdx'].str[3:7].astype(int)
df_vdx['accnb_vdx'] = [1000000000 * (row['year_reported_vdx'] % 2010)
                       + row['go_num_vdx'] for i, row in df_vdx.iterrows()]
df_vdx

,go_id_vdx,go_num_vdx,date_vdx,year_vdx,year_reported_vdx,accnb_vdx
0,GO-20141259736,1259736,2014-01-01,2014,2014,4001259736
1,GO-20141259991,1259991,2014-01-01,2014,2014,4001259991
2,GO-20141260385,1260385,2014-01-01,2014,2014,4001260385
3,GO-20141260478,1260478,2014-01-01,2014,2014,4001260478
4,GO-20141260499,1260499,2014-01-01,2014,2014,4001260499
...,...,...,...,...,...,...
75137,GO-20192521610,2521610,2019-12-31,2019,2019,9002521610
75138,GO-20192522200,2522200,2019-12-31,2019,2019,9002522200
75139,GO-20192522289,2522289,2019-12-31,2019,2019,9002522289
75140,GO-20192522442,2522442,2019-12-31,2019,2019,9002522442


In [16]:
dups = df_vdx.groupby('go_num_vdx')['go_id_vdx'].count()
dups[dups > 1]

go_num_vdx
1242       2
4192       2
5122       2
9508       2
9770       2
          ..
3164556    2
3178179    2
3179148    2
3242962    2
3250209    2
Name: go_id_vdx, Length: 789, dtype: int64

So we can't join based on the GO number suffix alone - we need to join using the full ID, which includes year, or join on both year and GO number, or ACCNB.

## Merge Datasets Together

For this three-way comparision, we'll drop all the Oracle collisions that look like they're from the CRC, since we wouldn't expect them to be in the other two datasets anyway.

We start by merging the XML and Oracle data on ACC number alone:

In [17]:
df_merge_test = pd.merge(coln_xml_20152019, df_oracle.loc[df_oracle['source_ocl'] != 'CRC', :],
                         how='outer', left_on='accnb_xml', right_on='accnb_ocl',
                         suffixes=('_xml', '_ocl'))

In [18]:
df_merge_test.loc[~df_merge_test['year_reported_xml'].isna()
                  & ~df_merge_test['year_ocl'].isna()
                  & (df_merge_test['year_reported_xml'] != df_merge_test['year_ocl']), :]

,year_reported_xml,go_num_xml,accnb_xml,accnb_ocl,year_ocl,date_ocl,source_ocl,table_ocl
16820,2015.0,1076024.0,5.001076e+09,5.001076e+09,2014.0,2014-12-03,TPS,ACC
16916,2015.0,1086907.0,5.001087e+09,5.001087e+09,2014.0,2014-12-02,TPS,ACC
17877,2015.0,1216302.0,5.001216e+09,5.001216e+09,2014.0,2014-07-16,TPS,ACC
24643,2015.0,2300393.0,5.002300e+09,5.002300e+09,2014.0,2014-06-16,TPS,ACC
25023,2015.0,279522.0,5.000280e+09,5.000280e+09,2014.0,2014-02-16,TPS,ACC
25340,2015.0,311757.0,5.000312e+09,5.000312e+09,2014.0,2014-12-10,TPS,ACC
25371,2015.0,31625.0,5.000032e+09,5.000032e+09,2014.0,2014-01-06,TPS,ACC
25645,2015.0,3492151.0,5.003492e+09,5.003492e+09,2014.0,2014-12-15,TPS,ACC
25673,2015.0,3523719.0,5.003524e+09,5.003524e+09,2014.0,2014-12-20,TPS,ACC
26436,2015.0,47040.0,5.000047e+09,5.000047e+09,2014.0,2014-12-08,TPS,ACC


There's around 50 cases where the **reported date** (which makes up the first four digits of the GO number) is not the same as the **occurrence date** (which is stored in `ACCDATE` and is therefore what's reported in `year_ocl`). Because of this it's difficult for us to merge on both ACC number and year, so we'll stick with ACC number. This produces two duplicates, which I guess we'll live with:

In [19]:
dups = df_merge_test.groupby(['accnb_ocl'])['accnb_ocl'].count()
dups[dups > 1].index

Float64Index([5000009168.0, 5000031625.0], dtype='float64', name='accnb_ocl')

We can either try merging VDX with Oracle or with XML. The logical thing to try first is XML, since that is the order of operations in the data pipeline (VDX is dumped to our FTP server as XMLs, which are then uploaded into Oracle). The alternative, joining directly on Oracle, would let us determine if there's a secret additional stash of XML files at another location.

To join VDX and XML, we can try joining directly to year reported and GO number.

In [20]:
df_merge_test = pd.merge(df_vdx, coln_xml_20152019,
                         how='outer', left_on=('year_reported_vdx', 'go_num_vdx'),
                         right_on=('year_reported_xml', 'go_num_xml'),
                         suffixes=('_vdx', '_xml'))

Check we haven't generated duplicates.

In [21]:
duptest = df_merge_test.groupby(['year_reported_vdx', 'go_num_vdx'])['year_reported_vdx'].count()
duptest[duptest > 1]

Series([], Name: year_reported_vdx, dtype: int64)

In [22]:
duptest = df_merge_test.groupby(['year_reported_xml', 'go_num_xml'])['year_reported_xml'].count()
duptest[duptest > 1]

Series([], Name: year_reported_xml, dtype: int64)

In [23]:
df_merge_test['join_status'] = 'Both'
df_merge_test.loc[df_merge_test['year_reported_xml'].isna(), 'join_status'] = 'VDX Only'
df_merge_test.loc[df_merge_test['year_reported_vdx'].isna(), 'join_status'] = 'XML Only'

df_merge_test['join_status'].value_counts()

Both        73278
XML Only     7542
VDX Only     1864
Name: join_status, dtype: int64

A number of XML files reference GO numbers that aren't in the VDX, and vice versa. If we loosened our requirements

In [24]:
coln_xml_20152019.head()

,year_reported_xml,go_num_xml,accnb_xml
1537,2014,1008236,4001008236
1538,2014,1057534,4001057534
1539,2014,1059448,4001059448
1540,2014,10667333,4010667333
1541,2014,1071106,4001071106


In [25]:
df_vdx_notmerged = df_merge_test.loc[df_merge_test['join_status'] == 'VDX Only',
                                     ['go_id_vdx', 'go_num_vdx', 'date_vdx',
                                      'year_vdx', 'year_reported_vdx']].copy()

df_xml_notmerged = df_merge_test.loc[df_merge_test['join_status'] == 'XML Only',
                                     ['year_reported_xml', 'go_num_xml', 'accnb_xml']].copy()

df_merge_test2 = pd.merge(df_vdx_notmerged, df_xml_notmerged,
                          how='outer', left_on='go_num_vdx',
                          right_on='go_num_xml',
                          suffixes=('_vdx', '_xml'))

df_merge_test2.loc[~df_merge_test2['year_reported_vdx'].isna()
                   & ~df_merge_test2['year_reported_xml'].isna(), :]

,go_id_vdx,go_num_vdx,date_vdx,year_vdx,year_reported_vdx,year_reported_xml,go_num_xml,accnb_xml
141,GO-20142953887,2953887.0,2014-09-21,2014.0,2014.0,2015.0,2953887.0,5.002954e+09
264,GO-20152226570,2226570.0,2015-12-22,2015.0,2015.0,2014.0,2226570.0,4.002227e+09
430,GO-2017211194,211194.0,2017-02-03,2017.0,2017.0,2015.0,211194.0,5.000211e+09
1182,GO-201942819,42819.0,2019-01-08,2019.0,2019.0,2015.0,42819.0,5.000043e+09
1225,GO-2019143566,143566.0,2019-01-23,2019.0,2019.0,2015.0,143566.0,5.000144e+09


So even if we only merged on GO number, we'd only account for 5 additional events, and their reported years really don't match in some cases.

What about merging direction with Oracle?

In [26]:
df_merge_test = pd.merge(df_vdx, df_oracle.loc[df_oracle['source_ocl'] != 'CRC', :],
                         how='outer', left_on=('year_vdx', 'accnb_vdx'),
                         right_on=('year_ocl', 'accnb_ocl'),
                         suffixes=('_vdx', '_ocl'))

In [27]:
duptest = df_merge_test.groupby(['year_vdx', 'accnb_vdx'])['year_vdx'].count()
duptest[duptest > 1]

Series([], Name: year_vdx, dtype: int64)

In [28]:
duptest = df_merge_test.groupby(['year_ocl', 'accnb_ocl'])['year_ocl'].count()
duptest[duptest > 1]

Series([], Name: year_ocl, dtype: int64)

In [29]:
df_merge_test['join_status'] = 'Both'
df_merge_test.loc[df_merge_test['year_ocl'].isna(), 'join_status'] = 'VDX Only'
df_merge_test.loc[df_merge_test['year_vdx'].isna(), 'join_status'] = 'Oracle Only'

df_merge_test['join_status'].value_counts()

Both           73107
Oracle Only     7611
VDX Only        2035
Name: join_status, dtype: int64

In [30]:
df_merge_test.loc[~df_merge_test['accnb_ocl'].isna(), 'table_ocl'].value_counts()

ACC            71939
ACC_ARCHIVE     8779
Name: table_ocl, dtype: int64

A couple of observations here - there are considerably more Oracle-only collisions than Versadex only collisions. Of the matched collisions, approximately 9000 are in `ACC_ARCHIVE` (and we're dropping duplicates, but there are dozens, not thousands, of those).

## Final Results

### Two-Way Join Results

These are finalized versions of the results above.

#### Versadex-XML

In [31]:
df_vdxxml = pd.merge(df_vdx, coln_xml_20152019,
                     how='outer', left_on=('year_reported_vdx', 'go_num_vdx'),
                     right_on=('year_reported_xml', 'go_num_xml'),
                     suffixes=('_vdx', '_xml'))

df_vdxxml['join_status'] = 'Both'
df_vdxxml.loc[df_vdxxml['year_reported_xml'].isna(), 'join_status'] = 'VDX Only'
df_vdxxml.loc[df_vdxxml['year_reported_vdx'].isna(), 'join_status'] = 'XML Only'

In [32]:
df_vdxxml['join_status'].value_counts()

Both        73278
XML Only     7542
VDX Only     1864
Name: join_status, dtype: int64

#### XML-Oracle

In [33]:
df_xmlocl = pd.merge(coln_xml_20152019, df_oracle.loc[df_oracle['source_ocl'] != 'CRC', :],
                     how='outer', left_on='accnb_xml', right_on='accnb_ocl',
                     suffixes=('_xml', '_ocl'))

df_xmlocl['join_status'] = 'Both'
df_xmlocl.loc[df_xmlocl['accnb_xml'].isna(), 'join_status'] = 'Oracle Only'
df_xmlocl.loc[df_xmlocl['accnb_ocl'].isna(), 'join_status'] = 'XML Only'

In [34]:
df_xmlocl['join_status'].value_counts()

Both           80369
XML Only         453
Oracle Only      349
Name: join_status, dtype: int64

In [35]:
duptest1 = df_xmlocl.groupby('accnb_xml')['accnb_xml'].count()
duptest2 = df_xmlocl.groupby('accnb_ocl')['accnb_ocl'].count()
set(duptest1[duptest1 > 1].index.astype(int)).union(
    set(duptest2[duptest2 > 1].index.astype(int)))

{5000009168, 5000031625}

#### VDX-Oracle

In [36]:
df_vdxocl = pd.merge(df_vdx, df_oracle.loc[df_oracle['source_ocl'] != 'CRC', :],
                     how='outer', left_on=('year_vdx', 'accnb_vdx'),
                     right_on=('year_ocl', 'accnb_ocl'),
                     suffixes=('_vdx', '_ocl'))

df_vdxocl['join_status'] = 'Both'
df_vdxocl.loc[df_vdxocl['accnb_vdx'].isna(), 'join_status'] = 'Oracle Only'
df_vdxocl.loc[df_vdxocl['accnb_ocl'].isna(), 'join_status'] = 'VDX Only'

In [37]:
df_vdxocl['join_status'].value_counts()

Both           73107
Oracle Only     7611
VDX Only        2035
Name: join_status, dtype: int64

### Three-Way Join Results

Time for some Venn Diagram goodness.

#### VDX-XML-Oracle

This merges VDX with XML, and then merges the result with Oracle through XML columns.

In [38]:
df_step1 = pd.merge(df_vdx, coln_xml_20152019,
                    how='outer', left_on=('year_reported_vdx', 'go_num_vdx'),
                    right_on=('year_reported_xml', 'go_num_xml'),
                    suffixes=('_vdx', '_xml'))

df_vdx_xml_ocl_raw = pd.merge(df_step1, df_oracle.loc[df_oracle['source_ocl'] != 'CRC', :],
                              how='outer', left_on='accnb_xml', right_on='accnb_ocl',
                              suffixes=('_vdxxml', '_ocl'))

In [39]:
# There are some VDX and Oracle rows that could be merged together, but aren't captured above:
oracle_only = (df_vdx_xml_ocl_raw['accnb_vdx'].isna()
               & df_vdx_xml_ocl_raw['accnb_xml'].isna())
vdx_only = (df_vdx_xml_ocl_raw['accnb_xml'].isna()
            & df_vdx_xml_ocl_raw['accnb_ocl'].isna())
everything_else = ~oracle_only & ~vdx_only

df_vdx_xml_ocl_step2 = pd.merge(
    df_vdx_xml_ocl_raw.loc[
        vdx_only, ['go_id_vdx', 'go_num_vdx', 'date_vdx',
                   'year_vdx', 'year_reported_vdx',
                   'accnb_vdx', 'year_reported_xml',
                   'go_num_xml', 'accnb_xml']].drop_duplicates(),
    df_vdx_xml_ocl_raw.loc[
        oracle_only, ['accnb_ocl', 'year_ocl', 'date_ocl',
                      'source_ocl', 'table_ocl']].drop_duplicates(),
    how='outer', left_on=('year_vdx', 'accnb_vdx'),
    right_on=('year_ocl', 'accnb_ocl'),
    suffixes=('_vdx', '_ocl'))

df_vdx_xml_ocl = pd.concat([df_vdx_xml_ocl_raw.loc[everything_else, :],
                            df_vdx_xml_ocl_step2[df_vdx_xml_ocl_raw.columns]],
                           ignore_index=True)

In [40]:
df_vdx_xml_ocl['join_status'] = 'In All'

# This includes XML AND/OR Oracle, but we replace exclusive cases in the next code block.
df_vdx_xml_ocl.loc[df_vdx_xml_ocl['accnb_vdx'].isna(), 'join_status'] = 'XML & Oracle'
df_vdx_xml_ocl.loc[df_vdx_xml_ocl['accnb_xml'].isna(), 'join_status'] = 'VDX & Oracle'
df_vdx_xml_ocl.loc[df_vdx_xml_ocl['accnb_ocl'].isna(), 'join_status'] = 'VDX & XML'

df_vdx_xml_ocl.loc[(df_vdx_xml_ocl['accnb_vdx'].isna()
                    & df_vdx_xml_ocl['accnb_xml'].isna()), 'join_status'] = 'Oracle Only'
df_vdx_xml_ocl.loc[(df_vdx_xml_ocl['accnb_vdx'].isna()
                    & df_vdx_xml_ocl['accnb_ocl'].isna()), 'join_status'] = 'XML Only'
df_vdx_xml_ocl.loc[(df_vdx_xml_ocl['accnb_xml'].isna()
                    & df_vdx_xml_ocl['accnb_ocl'].isna()), 'join_status'] = 'VDX Only'

df_vdx_xml_ocl['year_bestguess'] = df_vdx_xml_ocl['year_vdx']
df_vdx_xml_ocl.loc[df_vdx_xml_ocl['year_bestguess'].isna(), 'year_bestguess'] = df_vdx_xml_ocl['year_ocl']
# Only thing we have when it's only XML
df_vdx_xml_ocl.loc[df_vdx_xml_ocl['year_bestguess'].isna(), 'year_bestguess'] = df_vdx_xml_ocl['year_reported_xml']
df_vdx_xml_ocl['year_bestguess'] = df_vdx_xml_ocl['year_bestguess'].astype(int)

In [41]:
df_vdx_xml_ocl.groupby(['year_bestguess', 'join_status'])['join_status'].count().unstack()

join_status,In All,Oracle Only,VDX & Oracle,VDX & XML,VDX Only,XML & Oracle,XML Only
year_bestguess,,,,,,,
2014,16117.0,15.0,NaN,61.0,181.0,2324.0,51.0
2015,14814.0,14.0,NaN,32.0,86.0,1703.0,10.0
2016,12080.0,36.0,41.0,62.0,108.0,900.0,12.0
2017,10426.0,31.0,45.0,87.0,261.0,820.0,10.0
2018,10057.0,23.0,52.0,50.0,400.0,841.0,5.0
2019,9429.0,43.0,49.0,65.0,641.0,858.0,8.0


In [42]:
df_vdx_xml_ocl.groupby(['year_bestguess', 'join_status'])['join_status'].count().unstack().sum(axis=0)

join_status
In All          72923.0
Oracle Only       162.0
VDX & Oracle      187.0
VDX & XML         357.0
VDX Only         1677.0
XML & Oracle     7446.0
XML Only           96.0
dtype: float64

In [43]:
df_vdx_xml_ocl.groupby(['year_bestguess', 'join_status'])['join_status'].count().unstack().sum().sum()

82848.0

#### Dump Simplified Matrix for TPS

In [44]:
acc_mtx = df_vdx_xml_ocl.groupby(['year_bestguess', 'join_status'])['join_status'].count().unstack(fill_value=0)
acc_mtx['TS Only'] = acc_mtx['Oracle Only'] + acc_mtx['XML & Oracle'] + acc_mtx['XML Only']
acc_mtx['Mixed'] = acc_mtx['VDX & Oracle'] + acc_mtx['VDX & XML']
acc_mtx[['In All', 'VDX Only', 'TS Only', 'Mixed']].to_csv('summary_fortps.csv')

### VDX-Oracle-XML

This merges VDX with Oracle, then Oracle with XML.

In [45]:
df_step1 = pd.merge(df_vdx, df_oracle.loc[df_oracle['source_ocl'] != 'CRC', :],
                    how='outer', left_on=('year_vdx', 'accnb_vdx'),
                    right_on=('year_ocl', 'accnb_ocl'),
                    suffixes=('_vdx', '_ocl'))

df_vdx_ocl_xml_raw = pd.merge(df_step1, coln_xml_20152019,
                              how='outer', left_on='accnb_ocl', right_on='accnb_xml',
                              suffixes=('_vdxocl', '_xml'))

In [46]:
# There are some VDX and XML rows that could be merged together, but aren't captured above:
xml_only = (df_vdx_ocl_xml_raw['accnb_vdx'].isna()
            & df_vdx_ocl_xml_raw['accnb_ocl'].isna())
vdx_only = (df_vdx_ocl_xml_raw['accnb_xml'].isna()
            & df_vdx_ocl_xml_raw['accnb_ocl'].isna())
everything_else = ~xml_only & ~vdx_only

df_vdx_ocl_xml_step2 = pd.merge(
    df_vdx_ocl_xml_raw.loc[
        vdx_only, ['go_id_vdx', 'go_num_vdx', 'date_vdx',
                   'year_vdx', 'year_reported_vdx',
                   'accnb_vdx']].drop_duplicates(),
    df_vdx_ocl_xml_raw.loc[
        xml_only, ['year_reported_xml', 'go_num_xml', 'accnb_xml',
                   'accnb_ocl', 'year_ocl', 'date_ocl',
                   'source_ocl', 'table_ocl']].drop_duplicates(),
    how='outer', left_on=('year_reported_vdx', 'go_num_vdx'),
    right_on=('year_reported_xml', 'go_num_xml'),
    suffixes=('_vdx', '_xml'))

df_vdx_ocl_xml = pd.concat([df_vdx_ocl_xml_raw.loc[everything_else, :],
                            df_vdx_ocl_xml_step2[df_vdx_ocl_xml_raw.columns]],
                           ignore_index=True)

In [47]:
df_vdx_ocl_xml['join_status'] = 'In All'

df_vdx_ocl_xml.loc[df_vdx_ocl_xml['accnb_vdx'].isna(), 'join_status'] = 'XML & Oracle'
df_vdx_ocl_xml.loc[df_vdx_ocl_xml['accnb_xml'].isna(), 'join_status'] = 'VDX & Oracle'
df_vdx_ocl_xml.loc[df_vdx_ocl_xml['accnb_ocl'].isna(), 'join_status'] = 'VDX & XML'

df_vdx_ocl_xml.loc[(df_vdx_ocl_xml['accnb_vdx'].isna()
                    & df_vdx_ocl_xml['accnb_xml'].isna()), 'join_status'] = 'Oracle Only'
df_vdx_ocl_xml.loc[(df_vdx_ocl_xml['accnb_vdx'].isna()
                    & df_vdx_ocl_xml['accnb_ocl'].isna()), 'join_status'] = 'XML Only'
df_vdx_ocl_xml.loc[(df_vdx_ocl_xml['accnb_xml'].isna()
                    & df_vdx_ocl_xml['accnb_ocl'].isna()), 'join_status'] = 'VDX Only'

df_vdx_ocl_xml['year_bestguess'] = df_vdx_ocl_xml['year_vdx']
df_vdx_ocl_xml.loc[df_vdx_ocl_xml['year_bestguess'].isna(), 'year_bestguess'] = df_vdx_ocl_xml['year_ocl']
df_vdx_ocl_xml.loc[df_vdx_ocl_xml['year_bestguess'].isna(), 'year_bestguess'] = df_vdx_ocl_xml['year_reported_xml']
df_vdx_ocl_xml['year_bestguess'] = df_vdx_ocl_xml['year_bestguess'].astype(int)

In [48]:
df_vdx_ocl_xml.groupby(['year_bestguess', 'join_status'])['join_status'].count().unstack(fill_value=0.)

join_status,In All,Oracle Only,VDX & Oracle,VDX & XML,VDX Only,XML & Oracle,XML Only
year_bestguess,,,,,,,
2014,16117.0,15.0,0.0,61.0,181.0,2326.0,51.0
2015,14812.0,14.0,0.0,32.0,86.0,1703.0,10.0
2016,12080.0,36.0,41.0,62.0,108.0,901.0,12.0
2017,10425.0,31.0,45.0,87.0,262.0,820.0,10.0
2018,10057.0,23.0,52.0,50.0,400.0,841.0,5.0
2019,9429.0,43.0,49.0,65.0,641.0,858.0,8.0


In [49]:
df_vdx_ocl_xml.groupby(['year_bestguess', 'join_status'])['join_status'].count().unstack(fill_value=0.).sum(axis=0)

join_status
In All          72920.0
Oracle Only       162.0
VDX & Oracle      187.0
VDX & XML         357.0
VDX Only         1678.0
XML & Oracle     7449.0
XML Only           96.0
dtype: float64

In [50]:
df_vdx_ocl_xml.groupby(['year_bestguess', 'join_status'])['join_status'].count().unstack(fill_value=0.).sum(axis=0).sum()

82849.0

The order of operations matters when conducting a three-way merge. This is because there are possible connections between Oracle and VDX that don't include XML, and XML and VDX that don't include Oracle. Since we're performing an outer join, the size of `df_step1` depends on which of these three datasets we join together first. The solution is to perform a self-join those events that had no prior opportunity to join: VDX and Oracle for `df_vdx_xml_ocl`, and VDX and XML for `df_vdx_ocl_xml`. Once this is done, `df_vdx_xml_ocl` and `df_vdx_ocl_xml` are within a few events of one another (this is likely due to when duplicates are generated during merges.)

Since these results are so close to one another, we'll just take `df_vdx_xml_ocl` as the canonical results. 

Of the XML & Oracle-only events, how many are from `ACC_ARCHIVE`?

In [51]:
df_vdx_xml_ocl.loc[df_vdx_xml_ocl['join_status'] == 'XML & Oracle', 'table_ocl'].value_counts()

ACC            6560
ACC_ARCHIVE     886
Name: table_ocl, dtype: int64

### Observations

These numbers are +/- 2 due to duplicates generated during table merges.

- There are 82,848 unique entries across the three databases. Of these, 72,923 exist in all three databases.
- 7446 events only exist in (both) the XML and Oracle databases. When broken up by year, we see that more of these come from 2014 than 2019. There are three possible explanations I can think of off the top of my head:
  - These are actually from the CRC, but were placed in the TPS folders by mistake. (This assumes the CRC XML format is identical to the TPS one.)
  - These XMLs represent collision events that should have been merged with other events, but weren't done so in Oracle for some reason. Of the 7446 events, 6560 are in `ACC`, not `ACC_ARCHIVE`, so they should have been merged by a mail clerk if they represent the same event as some other row in `ACC`.
  - These collisions weren't included in TPS's data dump to us, for some reason.
- 1678 events only exist in Versadex. When broken up by year, we see a steady increase from year to year, with the plurality of these events coming from 2019. Possible explanations:
  - These collisions were unsuccessfully transmitted to us. This is pretty likely.
  - These collisions represent duplicates in some way (eg. two 911 calls for the same incident that weren't unified in TPS's database).
  - The XMLs were successfully transferred but to a folder not connected to the ACC loading process.
- There are 96 XML-only files. These may have been merged with other reports in both TPS and ACC databases.
- There are 162 Oracle-only files, which may either be miscalssified CRC collisions or manually generated by the mail clerks.
- There are 357 VDX and XML files that don't show up in either `ACC` or `ACC_ARCHIVE`. These may have not been successfully loaded by Jim Millington's process.
- There are 187 VDX & Oracle-only files. These are weird - I can only surmise some XMLs may have accidentally been deleted?

## Load ACC Data from Oracle

In [52]:
sql_query = """SELECT ACCNB,
       ACCDATE,
       ACCTIME,
       STNAME1,
       STNAME2,
       ACCLASS,
       ACCLOC,
       LOCCOORD,
       LONGITUDE + 0.00021 lon,
       LATITUDE + 0.000045 lat,
       INVTYPE,
       INJURY,
       POLICE_AGENCY,
       CRC_UNIT
FROM TRAFFIC.ACC__ALL
WHERE accdate BETWEEN TO_DATE('2014-01-01', 'YYYY-MM-DD') AND TO_DATE('2019-12-31', 'YYYY-MM-DD')"""

with cx_Oracle.connect(oracle_settings['user'], oracle_settings['password'],
                       "{host}:{port}/{service_name}".format(**oracle_settings),
                       encoding="UTF-8") as conn:
    df_accdata_raw = pd.read_sql(sql_query, con=conn)
df_accdata_raw['ACCNB'] = df_accdata_raw['ACCNB'].astype(int)
df_accdata_raw['ACCDATE'] = pd.to_datetime(df_accdata_raw['ACCDATE'])
df_accdata_raw['ACCYEAR'] = df_accdata_raw['ACCDATE'].dt.year

In [53]:
# Get join status for these ACCNBs.
df_accdata_js = df_vdx_xml_ocl[['accnb_ocl', 'year_ocl', 'join_status']].copy().dropna()
df_accdata_js['accnb_ocl'] = df_accdata_js['accnb_ocl'].astype(int)
df_accdata_js['year_ocl'] = df_accdata_js['year_ocl'].astype(int)

df_accdata = pd.merge(df_accdata_js, df_accdata_raw,
                      how='left',
                      left_on=('accnb_ocl', 'year_ocl'),
                      right_on=('ACCNB', 'ACCYEAR'))

In [54]:
sql_query = """SELECT * FROM collision_factors.acclass;"""

with psycopg2.connect(**postgres_settings) as db_con:
    df_acclass = pd.read_sql(sql_query, db_con)
    
    
df_collision_type = pd.merge(df_accdata[['ACCNB', 'ACCYEAR', 'ACCLASS']], df_acclass,
                             how='left', left_on='ACCLASS',
                             right_on='acclass')
df_collision_type = df_collision_type[['ACCNB', 'ACCYEAR', 'description']]
df_collision_type.columns = ['ACCNB', 'ACCYEAR', 'collision_type']

In [55]:
df_accdata = pd.merge(df_accdata, df_collision_type,
                      how='left',
                      left_on=('ACCNB', 'ACCYEAR'),
                      right_on=('ACCNB', 'ACCYEAR'))

In [56]:
df_accdata.groupby(['join_status', 'collision_type'])['ACCNB'].nunique().unstack(fill_value=0)

collision_type,Fatal,Non-Fatal Injury,Non-Reportable,Other,Property Damage Only
join_status,,,,,
In All,117,43521,304,16,28873
Oracle Only,28,27,0,1,106
VDX & Oracle,186,1,0,0,0
XML & Oracle,12,4277,84,0,3044


In [104]:
def conv_injury_to_float(x):
    try:
        return int(x)
    except:
        return -1

df_accdata['INJURY_INT'] = df_accdata['INJURY'].apply(conv_injury_to_float)

df_maxinj = df_accdata.groupby(['ACCNB', 'ACCYEAR'])[['join_status', 'INJURY_INT']].max()

In [105]:
sql_query = """SELECT * FROM collision_factors.injury;"""

with psycopg2.connect(**postgres_settings) as db_con:
    df_invinjclass = pd.read_sql(sql_query, db_con)
    
df_invinjclass = df_invinjclass[['injury', 'description']].dropna()
df_invinjclass.columns = ['INJURY', 'involved_injury']
df_invinjclass.loc[0, 'involved_injury'] = 'No Injury'
df_invinjclass['INJURY'] = df_invinjclass['INJURY'].astype(int)

df_maxinj = pd.merge(df_maxinj, df_invinjclass,
                     how='left', left_on='INJURY_INT',
                     right_on='INJURY')

df_maxinj.rename(columns={'involved_injury': 'max_involved_injury'}, inplace=True)

In [107]:
df_maxinj.groupby(['join_status', 'max_involved_injury'])['join_status'].count().unstack(fill_value=0)

max_involved_injury,Fatal,Major,Minimal,Minor,No Injury
join_status,,,,,
In All,117,1853,18783,22472,29418
Oracle Only,28,1,13,13,107
VDX & Oracle,186,1,0,0,0
XML & Oracle,12,166,2431,1630,3179


## Upload XML + Oracle `ACC` Numbers to Postgres

For ease of analysis, and for an easy way to transmit data to Andrew, let's dump the `ACCNB` and `YEAR` of `ACC` collisions in both the XML and Oracle, but not in the TPS dump.

In [109]:
df_export = df_vdx_xml_ocl.loc[(df_vdx_xml_ocl['join_status'] == 'XML & Oracle')
                               & (df_vdx_xml_ocl['table_ocl'] == 'ACC'), ['year_ocl', 'accnb_ocl']]
df_export.reset_index(inplace=True, drop=True)
df_export['year_ocl'] = df_export['year_ocl'].astype(int)
df_export['accnb_ocl'] = df_export['accnb_ocl'].astype(int)

In [110]:
pgengine = sqla.create_engine("postgresql://{user}:{password}@{host}:{port}/{dbname}"
                              .format(**postgres_settings))
df_export.to_sql('coln_not_in_tps', con=pgengine,
                 if_exists='replace', schema='czhu')
pgengine.dispose()

## Dump VDX vs. XML comparison for TPS

In [173]:
coln_xml_wtfile = pd.merge(coln_xml_20152019[['year_reported_xml', 'go_num_xml']],
                           coln_xml_r[['filename', 'repyear', 'go_number']],
                           how='left', left_on=('year_reported_xml', 'go_num_xml'),
                           right_on=('repyear', 'go_number'))[['year_reported_xml', 'go_num_xml', 'filename']]

In [174]:
df_vx_tps = pd.merge(df_vdx_xml_ocl[['go_id_vdx', 'year_reported_xml', 'go_num_xml',
                                     'accnb_ocl', 'year_ocl', 'join_status']],
                     coln_xml_wtfile,
                     how='left', left_on=('year_reported_xml', 'go_num_xml'),
                     right_on=('year_reported_xml', 'go_num_xml'),
                     suffixes=('_full', '_xml'))

In [175]:
df_vx_tps.loc[df_vx_tps['join_status'].isin(['In All', 'VDX & XML', 'VDX & Oracle']), 'join_status'] = 'In Both'
df_vx_tps.loc[df_vx_tps['join_status'].isin(['XML Only', 'XML & Oracle', 'Oracle Only']), 'join_status'] = 'TS Only'

In [176]:
df_vx_tps['filename'].fillna('Not Found', inplace=True)
df_vx_tps['go_id_vdx'].fillna('Not Found', inplace=True)
df_vx_tps['year_reported_xml'] = df_vx_tps['year_reported_xml'].fillna(-1).astype(int)
df_vx_tps['go_num_xml'] = df_vx_tps['go_num_xml'].fillna(-1).astype(int)
df_vx_tps['go_num_ocl'] = df_vx_tps['accnb_ocl'] - 1000000000 * (df_vx_tps['accnb_ocl'] // 1000000000)
df_vx_tps['accnb_ocl'] = df_vx_tps['accnb_ocl'].fillna(-1).astype(int)
df_vx_tps['go_num_ocl'] = df_vx_tps['go_num_ocl'].fillna(-1).astype(int)
df_vx_tps['year_ocl'] = df_vx_tps['year_ocl'].fillna(-1).astype(int)

In [184]:
df_vx_tps.rename(columns={'filename': 'xml_filename'}, inplace=True)
df_vx_tps = df_vx_tps[['go_id_vdx', 'xml_filename', 'year_reported_xml',
                       'go_num_xml', 'year_ocl', 'accnb_ocl', 'go_num_ocl',
                       'join_status']]

In [187]:
df_vx_tps.to_csv('/mnt/c/Users/czhu5/Documents/Ubuntu/20210520_vdxvstscomp_all.csv')

In [188]:
df_vx_tps.loc[df_vx_tps['join_status'] == 'TS Only', :].to_csv(
    '/mnt/c/Users/czhu5/Documents/Ubuntu/20210520_vdxvstscomp_tsonly.csv')

In [189]:
df_vx_tps.loc[df_vx_tps['join_status'] == 'VDX Only', :].to_csv(
    '/mnt/c/Users/czhu5/Documents/Ubuntu/20210520_vdxvstscomp_vdxonly.csv')